In [9]:
import pandas as pd
import json as js
import requests
from io import StringIO
from bs4 import BeautifulSoup

This section is what I tried in order to get the Billboard tables scraped. Because it's a complicated HTML structure (at least compared to Wikipedia) I had some trouble targeting it. I was getting errors until I spent an hour watching Beautifulsoup tutorials to try and target it, but decided it's worth it to explore other options first. If I end up needing this data I could give it another shot.

My (rough) idea now is to look at the number of times the top tours (maybe top five or so?) visited different cities and find which ones are the most visited. I think I would limit my analysis to cities in the US for the sake of brevity and also to ensure I have the right financial data. Something to consider would be the populations of the states as well, and how that could affect the number of times a city is visited. 

I'd also like to look into some kind of way to map the region of a city/venue. In my experience, artists will have two shows in California (North and South) and then a show in Chicago and Philidelphia and nothing in-between. Because of the population density, Chicago is like the bastion for concerts in the Midwest. It would be interesting to see region (and population) taken into account too to see how many are in each region.

In [10]:
url_wiki = 'https://www.billboard.com/2023-year-end-boxscore-charts/#pmc-protected-embed-2'

In [11]:
response1 = requests.get(url_wiki)
response1.status_code

403

In [12]:
soup = BeautifulSoup(response1.text, "html.parser")
the_table = soup.find('table',{'class':"medium datawrapper-OPu6B-sqmikr svelte-1smti70 striped compact resortable"})
print(the_table)

None


In [13]:
eras_tour = 'https://en.wikipedia.org/wiki/The_Eras_Tour#:~:text=Long%20Story%20Short%22-,Tour%20dates,-List%20of%202023'

In [14]:
response2 = requests.get(eras_tour)
response2.status_code

200

In [15]:
eras_tour_list = pd.read_html(StringIO(response2.text))
type(eras_tour_list)

list

I was a little surprised by how many things on Wikipedia pages are coded as tabled so it was mostly trial and error in terms of the index number to get the one I needed.

It was mentioned that the Wikipedia articles all cite data for their tables, but from my poking around it looks like they are once again citing Pollstar (paid) or an already put-together report, the same problems I've been running into. At least it's already concantenated for me though!

In [16]:
eras_2023 = eras_tour_list[5]
eras_2024 = eras_tour_list[6]

eras_2023.head()

,Date (2023),City,Country,Venue,Opening acts,Attendance,Revenue
0,March 17,Glendale,United States,State Farm Stadium,Paramore Gayle,—,—
1,March 18,Glendale,United States,State Farm Stadium,Paramore Gayle,—,—
2,March 24,Paradise[i],United States,Allegiant Stadium,Beabadoobee Gayle,—,—
3,March 25,Paradise[i],United States,Allegiant Stadium,Beabadoobee Gayle,—,—
4,March 31,Arlington,United States,AT&T Stadium,Muna Gayle,—,—


In [27]:
eras_tour_merged = pd.concat([eras_2023, eras_2024], axis = 0)
# eras_tour_merged

Something I noticed with the Eras tour here as well is that very few dates are in the US (comparatively). Just skimming the Wikipedia article it looked a lot more global than it actually was, but nearly half of the dates were in the US. Glad I checked the numbers first before worrying about the results.

In [18]:
count_us_eras = (eras_tour_merged['Country'] == 'United States').sum()
print("Times in US:", count_us_eras)

count_global_eras = eras_tour_merged['Country'].ne('United States').sum()
print("Times Abroad:", count_global_eras)

Times in US: 62
Times Abroad: 88


In [19]:
ej = 'https://en.wikipedia.org/wiki/Farewell_Yellow_Brick_Road'

In [20]:
response3 = requests.get(ej)
response3.status_code

200

In [21]:
ej_list = pd.read_html(StringIO(response3.text))
type(ej_list)

list

In [22]:
ej_tours = ej_list[1]
ej_tours.head()

,Date,City,Country,Venue,Attendance,Revenue
,North America[32][33][34],North America[32][33][34],North America[32][33][34],North America[32][33][34],North America[32][33][34],North America[32][33][34]
0,8 September 2018,Allentown,United States,PPL Center,"8,983 / 8,983","$1,492,859"
1,11 September 2018,Philadelphia,United States,Wells Fargo Center,"29,531 / 29,531","$4,273,021"
2,12 September 2018,Philadelphia,United States,Wells Fargo Center,"29,531 / 29,531","$4,273,021"
3,15 September 2018,Buffalo,United States,KeyBank Center,"15,581 / 15,641","$2,033,001"
4,16 September 2018,University Park,United States,Bryce Jordan Center,"12,421 / 12,421","$1,662,468"


So I'd have to do some work to see if this table is busted (I don't think so, but the North America being included in the header is weirding me out. It's a subheader on the page, I just don't know if it would mess up any of the other data), but this seemed a lot more US-heavy looking at the page.

Something I'm concerned about is that this tour ended in the first half of 2023. Would I use the data from 2018-2022 if I was using financial data from 2023? Most of the 2023 dates are outside of the US, anyways. There's a chance I may not use financial data anyways and instead use regional and financial, but still something to think about.

In [23]:
count_us_ej = (ej_tours['Country'] == 'United States').sum()
print("Times in US:", count_us_ej)

count_global_ej = ej_tours['Country'].ne('United States').sum()
print("Times Abroad:", count_global_ej)

Times in US: North America[32][33][34]    163
dtype: int64
Times Abroad: North America[32][33][34]    176
dtype: int64


The Eras is #1 on the list of top-grossing tours and Elton John is #3. Since all of the other pages follow the same format I'll skip adding them for now, but as of writing I'm planning on doing the top 5 or maybe top 10 of these datasets.

In [26]:
df = pd.read_excel(r'C:\Users\gamec\Downloads\q3t3.xlsx')
df.head()

,TABLE 3. STATE GOVERNMENT TAX COLLECTIONS BY STATE AND TYPE OF TAX,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data shown in thousands of dollars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Year 2023, Quarter 3 (July, August, September)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tax Description,Code,"U.S. Total (excludes Washington, D.C.)",NaN,Alabama*,NaN,Alaska*,NaN,Arizona*,NaN,...,Washington*,NaN,West Virginia*,NaN,Wisconsin*,NaN,Wyoming*,NaN,"Washington, D.C.",NaN


Ughh, for readibility (I think) the excel sheet has blank cells on either side of the numbers. Unsure how to remove that logically but I'm sure there's an easy way.

So, now the idea is to get all of the dataframes in, add the artists/tour to each one, concantenate them into one big dataset(?), remove all the tour stops that aren't from the United States, group them by city, manually add the states for each city (scripted since there are duplicates), and then begin running some data from there. The verdict is out on whether I want to add the region too. We'll see :)

Maybe worth noting I haven't paid more than $20 for a concert ticket in 10 years. This is a foreign world to me lol